Blurb from proposal : In a multi-agent system individual agents with different personas can communicate, collaborate, add additional context, and do work in parallel. They often have special rules for how to work as a team, decide things together, and handle disagreements.

Disclaimer (We can Remove or modify this once we rename the agents to build on our complex usecase): This notebooks uses and explains code snippets from Autogen Official Docs to demonstrate various Multi-Agent Usecases along with important concepts. Please refer to https://microsoft.github.io/autogen/docs/ for information.

In [ ]:
#Install packages
!pip install pyautogen
!pip install openai

In [ ]:
#Initilize the OpenAI API KEY and other Parameters
# TEAM CAN DECIDE WHICH ONE TO USE, EITHER OPENAI OR AZURE OPENAI.
%env OPENAI_API_KEY="REPLACE_ME"

In [ ]:
# Construct a Single Agent and Run it

In [ ]:
# Construct a Single Agent with Persona and Run it

In [ ]:
# Construct a Single Agent with a tool and a Persona and Run it

# Two-agent chat: the simplest form of conversation pattern where two agents chat with each other using Autogen.

In [ ]:
import os

from autogen import ConversableAgent

student_agent = ConversableAgent(
    name="Student_Agent",
    system_message="You are a student willing to learn.",
    llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]},
)
teacher_agent = ConversableAgent(
    name="Teacher_Agent",
    system_message="You are a math teacher.",
    llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

chat_result = student_agent.initiate_chat(
    teacher_agent,
    message="What is triangle inequality?",
    summary_method="reflection_with_llm",
    max_turns=2,
)

Let's look at the summary. It is contained within the chat_result object of type ChatResult, which was returned by the initiate_chat method.

In [ ]:
print(chat_result.summary)

In the given example, the summary method is configured to reflection_with_llm, which processes a list of conversation messages and summarizes them by invoking an LLM. Initially, the summary method attempts to utilize the recipient’s LLM; if it is unavailable, it defaults to the sender’s LLM. Here, the recipient is “Teacher_Agent” and the sender is “Student_Agent”. The input prompt for the LLM is the following default prompt:

In [ ]:
print(ConversableAgent.DEFAULT_SUMMARY_PROMPT)

# Sequential chat: 
a series of conversations between two agents, linked by a carryover mechanism that transfers the summary of the prior chat to the context of the subsequent chat.

In [ ]:
# The Number Agent always returns the same numbers.
number_agent = ConversableAgent(
    name="Number_Agent",
    system_message="You return me the numbers I give you, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Adder Agent adds 1 to each number it receives.
adder_agent = ConversableAgent(
    name="Adder_Agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Multiplier Agent multiplies each number it receives by 2.
multiplier_agent = ConversableAgent(
    name="Multiplier_Agent",
    system_message="You multiply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Subtracter Agent subtracts 1 from each number it receives.
subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Divider Agent divides each number it receives by 2.
divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

In [ ]:
# Start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat method.
chat_results = number_agent.initiate_chats(
    [
        {
            "recipient": adder_agent,
            "message": "14",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": multiplier_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": subtracter_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": divider_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
    ]
)

In [ ]:
print("First Chat Summary: ", chat_results[0].summary)
print("Second Chat Summary: ", chat_results[1].summary)
print("Third Chat Summary: ", chat_results[2].summary)
print("Fourth Chat Summary: ", chat_results[3].summary)

TODO : ADD EXAMPLE OF How to use a tool with an agent in Autogen? Probably Websearch using Tavily or Bing search API keys. More info here : https://microsoft.github.io/autogen/docs/tutorial/tool-use 

In [ ]:
TODO : ADD EXAMPLE OF How to use RAG with autogen Agent?

# Group Chat 
Up to this point, we have encountered conversation patterns that include either two agents or a series of two-agent interactions. AutoGen introduces a broader conversation model known as group chat, which includes more than two agents. The fundamental concept of group chat is that all agents participate in a unified conversation thread and share the same context. This approach is beneficial for tasks that necessitate collaboration among several agents.

Add Image Here from Autogen docs :


A group chat is orchestrated by a special agent type GroupChatManager. In the first step of the group chat, the Group Chat Manager selects an agent to speak. Then, the selected agent speaks and the message is sent back to the Group Chat Manager, who broadcasts the message to all other agents in the group. This process repeats until the conversation stops.

The Group Chat Manager can use several strategies to select the next agent. Currently, the following strategies are supported:

round_robin: The Group Chat Manager selects agents in a round-robin fashion based on the order of the agents provided.
random: The Group Chat Manager selects agents randomly.
manual: The Group Chat Manager selects agents by asking for human input.
auto: The default strategy, which selects agents using the Group Chat Manager’s LLM.

TODO : Use Group chat with auto strategy, define multiple agents all having access to web search tool or may be some other tool , each agent has independent persona and contributes to a group chat to create final report for user.


We can create 3 or more agents each having different personas and access to web search tool and a RAG tool? : 
for e.g, Economist persona
Environmentalist persona
Capatalist persona
May be add few more agents with different persona or having access to more tools.

Sample usecases : 
Impacts of asteroid impact on the world. 
Impacts of world war3 on the world. NOTE : We can change the usecase to something else but code structure can stay same.

Create final report for user on above usecase.

In [ ]:
# The `description` attribute is a string that describes the agent.
# It can also be set in `ConversableAgent` constructor.
adder_agent.description = "Add 1 to each input number."
multiplier_agent.description = "Multiply each input number by 2."
subtracter_agent.description = "Subtract 1 from each input number."
divider_agent.description = "Divide each input number by 2."
number_agent.description = "Return the numbers given."

from autogen import GroupChat
from autogen import GroupChatManager

group_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=6,
)
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

chat_result = number_agent.initiate_chat(
    group_chat_manager,
    message="My number is 3, I want to turn it into 13.",
    summary_method="reflection_with_llm",
)

In [ ]:
print(chat_result.summary)

If time and bandwidth permits,we can also do a combination of nested chat with group chat manager. 